In [40]:
pip install flash-attn --no-build-isolation

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 11.0 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.9.post1-cp310-cp310-linux_x86_64.whl size=120889689 sha256=5022ba11d48bf74926da9c16260f4ea2b9bb7f4e29bdb4bd6e1383ad1c55d16f
  Stored in directory: /home/ubuntu/.cache/pip/wheels/cc/ad/f6/7ccf0238790d6346e9fe622923a76ec218e890d356b9a2754a
Successfully built flash-attn

[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification

id2label = {
    0: "FIRST_PARTY_COLLECTION",
    1: "THIRD_PARTY_COLLECTION",
    2: "DATA_SECURITY",
    3: "DATA_RETENTION",
    4: "USER_ACCESS",
    5: "USER_CHOICE",
    6: "OTHER"
}

label2id = {
    "FIRST_PARTY_COLLECTION": 0 ,
    "THIRD_PARTY_COLLECTION": 1,
    "DATA_SECURITY": 2,
    "DATA_RETENTION": 3,
    "USER_ACCESS": 4,
    "USER_CHOICE": 5,
    "OTHER": 6
}

#config = PeftConfig.from_pretrained("shayan283/multilabel_classification")
model = AutoModelForSequenceClassification.from_pretrained("shayan283/multilabel_classification", num_labels=7,
            id2label=id2label, label2id=label2id, torch_dtype=torch.bfloat16, 
            attn_implementation="flash_attention_2")
# inference_model = PeftModel.from_pretrained(model, "shayan283/multilabel_classification")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("shayan283/multilabel_classification")
model.to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=Fal

In [11]:
import os
from transformers import PretrainedConfig, pipeline
text_classification_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)

RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [39]:
text = "Advertisers, measurement and other partners share information with us about you and the actions you have taken outside of the Platform, such as your activities on other websites and apps or in stores, including the products or services you purchased, online or in person."

predictions = text_classification_pipeline(text)
for prediction in predictions:
    print(f"Label: {prediction['label']}, Score: {prediction['score']:.4f}")

KeyboardInterrupt: 